In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
from models.generator import Generator
from models.discriminator import Discriminator
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
batch_size = 128
num_classes = 10
learning_rate = 0.002
num_epochs = 100
num_color_channels = 1
num_feature_maps_g = 32
num_feature_maps_d = 32
size_z = 100
adam_beta1 = 0.2
num_gpu = 0

In [ ]:
generator = Generator(size_z=size_z,
                      num_feature_maps=num_feature_maps_g,
                      num_color_channels=num_color_channels).to(device)
discriminator = Discriminator(num_feature_maps=num_feature_maps_d,
                              num_color_channels=num_color_channels).to(device)

In [8]:
generator.load_state_dict(torch.load('./saved_models/generator_100_epochs_mnist_excluded_3s.pkl', map_location=torch.device('cpu')))
discriminator.load_state_dict(torch.load('./saved_models/discriminator_100_epochs_mnist_excluded_3s.pkl', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
data = datasets.MNIST(
    root = "data",
    transform = transforms.ToTensor(),
    train=False
)

In [ ]:
def get_anomaly_score(x_query, g_z):
    x_prop = discriminator(x_query)
    g_z_prop = discriminator(g_z)

    loss_residual = torch.sum(torch.abs(x_query - g_z))
    loss_discriminative = torch.sum(torch.abs(x_prop - g_z_prop))

    return (1-.1)*loss_residual + .1* loss_discriminative

In [ ]:
figure = plt.figure(figsize=(5, 5))
cols, rows = 4, 4
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(data), size=(1,)).item()
    img, label = data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

In [ ]:
from torch.autograd import Variable
import torch.nn.init as init

start_idx = 64
test_size = 8

test_data_mnist = data.__dict__['data'][start_idx:start_idx+test_size]
test_data_mnist = test_data_mnist.view(test_size,1,28,28).type_as(torch.FloatTensor())
test_data_mnist.size()



In [ ]:
#z = Variable(init.normal(torch.zeros(test_size,100),mean=0,std=0.1),requires_grad=True)
z = torch.randn(8, size_z, 1, 1, device=device)
z_optimizer = torch.optim.Adam([z],lr=1e-4)

gen_fake = generator(z)
loss = get_anomaly_score(Variable(test_data_mnist),gen_fake)
# print(loss)

In [ ]:


for i in range(5000):
    gen_fake = generator(z)
    loss = get_anomaly_score(Variable(test_data_mnist),gen_fake)
    loss.backward()
    z_optimizer.step()

    if i%100==0:
        print(loss.data)
        '''
        target = test_data_mnist[1,0,:,:].numpy()
        plt.imshow(target,cmap="gray")
        plt.show()

        img=gen_fake.cpu().data[1,0,:,:].numpy()
        plt.imshow(img,cmap='gray')
        plt.show()
        '''

